In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

import gensim
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm
tqdm.pandas()

from unicode import join_jamos

In [2]:
DATA_TYPE = 'full-morphs'
DATA_NOSTOPS_TYPE = 'full-nouns'

In [3]:
DATA_DIR = 'lol'
TOKENIZING_DIR = '3-tokenizing'
EMBEDDING_DIR = '4-embedding'

tokenizing_dir = Path(TOKENIZING_DIR) / DATA_DIR
embedding_dir = Path(EMBEDDING_DIR) / DATA_DIR
embedding_dir.mkdir(parents=True, exist_ok=True)

### tokenize

In [4]:
df = pd.read_csv(tokenizing_dir / 'news-{}.csv'.format(DATA_TYPE)).dropna()

train_range = df.date.apply(lambda x: 20150000 < x < 20200000)
test_range = df.date.apply(lambda x: 20200000 < x < 20210000)

def my_train_test_split(data):
    return data[train_range], data[test_range]

def my_train_val_test_split(data):
    train, test = my_train_test_split(data)
    train, val = train_test_split(train, test_size=0.1, random_state=119)
    return train, val, test

contents = [x.split(' ') for x in my_train_val_test_split(df.contents)[0]]

### model

In [5]:
bigram = gensim.models.Phrases(contents)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[contents])
trigram_mod = gensim.models.phrases.Phraser(trigram)

### save

In [6]:
bigram_mod_path = embedding_dir / 'news-{}-bigram.bin'.format(DATA_TYPE)
bigram_mod.save(str(bigram_mod_path))

trigram_mod_path = embedding_dir / 'news-{}-trigram.bin'.format(DATA_TYPE)
trigram_mod.save(str(trigram_mod_path))

### load

In [7]:
bigram_mod_path = embedding_dir / 'news-{}-bigram.bin'.format(DATA_TYPE)
trigram_mod_path = embedding_dir / 'news-{}-trigram.bin'.format(DATA_TYPE)

bigram_mod = gensim.models.phrases.Phraser.load(str(bigram_mod_path))
trigram_mod = gensim.models.phrases.Phraser.load(str(trigram_mod_path))

def make_bigrams(texts):
    return [bigram_mod[doc] if doc is not np.nan else doc for doc in tqdm(texts)]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] if doc is not np.nan else doc for doc in tqdm(texts)]

### transform

In [8]:
ndf = pd.read_csv(tokenizing_dir / 'news-{}.csv'.format(DATA_NOSTOPS_TYPE))

title_trigrams = make_trigrams([x.split(' ') if x is not np.nan else x for x in ndf.title])
lede_trigrams = make_trigrams([x.split(' ') if x is not np.nan else x for x in ndf.lede])
contents_trigrams = make_trigrams([x.split(' ') if x is not np.nan else x for x in ndf.contents])

ndf.title = [' '.join(x) if x is not np.nan else x for x in title_trigrams]
ndf.lede = [' '.join(x) if x is not np.nan else x for x in lede_trigrams]
ndf.contents = [' '.join(x) if x is not np.nan else x for x in contents_trigrams]

In [9]:
[join_jamos(x) for x in ndf.title[1000:1010]]

['진에어 갱_플랭크 신경전',
 '진에어 년 테크_트리',
 '롤_드_컵 선발전 갱_맘 이창석 분위기_반전 진에어 세트 승리',
 '롤_드_컵 선발전 야스오 운영 세트',
 '롤_드_컵 선발전 마지막 교전 방 역전승 세트 선취',
 '롤_드_컵 선발전 진에어 날개 우승 롤_드_컵 숙원',
 '전투_민족 인_빅_터스 롤_드_컵 진출 번 팀',
 '감독 코멘트 이지훈_감독 팀 시너지 발휘',
 '팀 창단_후 롤_드_컵 진출 성공',
 '롤_드_컵 선발전 썸데이 김찬호 야스오 활약 세트 승리']

In [9]:
ndf.to_csv(embedding_dir / 'news-{}-trigram.csv'.format(DATA_NOSTOPS_TYPE), index=False)
ndf

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 00:49,20150801 00:49,1,1,핫_매치 리뷰 불량 학생 노동현 수능 대박 스토리,탈선 수능_만점 년 월 일 타이거즈 롤스터 정규_시즌 라운드 일 차 경기 팀 포스트_시즌,인벤,탈선 수능_만점 년 월 일 타이거즈 롤스터 정규_시즌 라운드 일 차 경기 팀 포스트...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,섬머 베누_소닉붐 뉴_클리어 사신 인터뷰 앞 승리,금일 일 롤 섬머_시즌 사상 스_베누_소닉붐 번 승리 기록 연패 번 승리 만큼 의미 감동,헝그리앱,금일 일 롤 섬머_시즌 사상 스_베누_소닉붐 번 승리 기록 연패 번 승리 만큼 의미...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,섬머 뱅 배준식 인터뷰 솔로_랭크 데 멘탈 중요,승리 소감 경기 오랜만 배성웅 벵 형 이지훈 이지훈 형 오랜만 때문 그 내,헝그리앱,승리 소감 경기 오랜만 배성웅 벵 형 이지훈 이지훈 형 오랜만 때문 그 내 편 다행...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,롬 섬머 베누_소닉붐 꼴찌 반격 개시 동안 설움 설욕 대망 승리,롤 섬머 경기 세트 기세 스_베누_소닉붐 승리 때문 진에어_그린_윙스 마지막 세트 ...,헝그리앱,롤 섬머 경기 세트 기세 스_베누_소닉붐 승리 때문 진에어_그린_윙스 마지막 세트 ...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,롬 섬머 베누_소닉붐 타_대승 기반 세트 승리,롤 섬머 경기 세트 시작 세트 때 진에어_그린_윙스 스_베누_소닉붐 세트 때 미드 ...,헝그리앱,롤 섬머 경기 세트 시작 세트 때 진에어_그린_윙스 스_베누_소닉붐 세트 때 미드 ...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,승강_전 다이브 승리 샌드박스 서라벌 세트 선승,일_종각 파크 섬머_스플릿 승강_전 최종_전 세트 샌드박스_게이밍 서라벌_게이밍 승...,인벤,일_종각 파크 섬머_스플릿 승강_전 최종_전 세트 샌드박스_게이밍 서라벌_게이밍 승...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,승강_전 서라벌 샌드박스 베스트_라인업 맞대결,서머 최종_전 대결 서라벌_게이밍 위 샌드박스_게이밍 사진 라이엇_게임즈 제공 마지...,데일리e스포츠,마지막 승강_전 승리 서라벌_게이밍 샌드박스_게이밍 승강_전 기용 베스트_라인업 출...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,일방 전투 승리 샌드박스_게이밍 서라벌_게이밍 상대 세트 승리,일 우리은행 승강_전 최종진_출전 샌드박스_게이밍 서라벌_게이밍 세트 경기 초반 기...,엑스포츠뉴스,일 우리은행 승강_전 최종진_출전 샌드박스_게이밍 서라벌_게이밍 세트 경기 초반 기...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,승강_전 샌드박스 카운터 기선_제압,용준 기자 최후 승강_전 샌드박스 다이브 공세 흐름 샌드박스 승강_전 최종_전 서전,OSEN,최후 승강_전 샌드박스 다이브 공세 흐름 샌드박스 승강_전 최종_전 서전 승리 장식...,0.0,490
